In [2]:
import numpy as np
from astropy.io import fits
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# Path to the FITS file
fits_file_path = 'space images/output_images_with_labels.fits'

# Lists to hold image data and labels
image_data_list = []
label_list = []

# Desired image size
# target_size = (256, 256)

# Open the FITS file
with fits.open(fits_file_path) as hdulist:
    # Iterate over each ImageHDU (starting from index 1)
    for hdu in hdulist[1:-1]:  # Skip the primary HDU (index 0) and the last HDU (label table)
        if isinstance(hdu, fits.ImageHDU):
            # Extract image data
            image_data = hdu.data
            
            # Convert the image data to PIL Image for resizing
            image = Image.fromarray(image_data)

            # Resize the image to 256x256
            # resized_image = image.resize(target_size)

            # Convert the resized image back to NumPy array
            resized_image_data = np.array(image)

            # Append the resized image data and corresponding label to the lists
            image_data_list.append(resized_image_data)
            label_list.append(hdu.header.get('OBJECT'))

# Convert the lists to NumPy arrays
image_data_array = np.array(image_data_list)
label_array = np.array(label_list)

# Encode labels as integers
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(label_array)

# Verify the shapes of the arrays
print(f"Resized image data array shape: {image_data_array.shape}")
# print(f"Encoded labels: {encoded_labels}")
print(f"Encoded labels: {label_array}")


Resized image data array shape: (1184, 128, 128, 3)
Encoded labels: ['cosmos space' 'cosmos space' 'cosmos space' ... 'stars' 'stars' 'stars']


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split


2024-09-05 01:01:00.427991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 01:01:00.569407: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 01:01:00.609747: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 01:01:00.823784: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 01:01:03.398954: W tensorflow/compiler/tf2

In [4]:

# Assuming `image_data_array` and `label_array` are already defined
# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_array)

# One-hot encode the labels for categorical cross-entropy loss
one_hot_labels = to_categorical(encoded_labels)

# image_data_array = np.expand_dims(image_data_array, axis=-1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(image_data_array, one_hot_labels, test_size=0.2, random_state=42)


In [5]:

# Load VGG16 model with pre-trained weights and exclude the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom top layers
x = base_model.output
x = Flatten()(x)
# x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(36, activation='relu')(x)
predictions = Dense(len(np.unique(encoded_labels)), activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


I0000 00:00:1725478267.937584  111166 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725478268.194389  111166 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725478268.194504  111166 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725478268.198696  111166 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725478268.198825  111166 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [6]:

# Prepare data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Define the callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Define batch size
batch_size = 4

# Train the model with data augmentation and callbacks
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          steps_per_epoch=len(X_train) // batch_size,
          epochs=25,
          validation_data=(X_test, y_test))
        #   callbacks=[early_stopping, model_checkpoint])


Epoch 1/25


/home/hardik/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1725478272.241123  111503 service.cc:146] XLA service 0x7f572000f3e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725478272.241292  111503 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-09-05 01:01:12.339993: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-05 01:01:12.819074: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


  4/236 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.1719 - loss: 2.3789     

I0000 00:00:1725478276.737146  111503 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


236/236 ━━━━━━━━━━━━━━━━━━━━ 26s 77ms/step - accuracy: 0.3355 - loss: 2.9137 - val_accuracy: 0.4810 - val_loss: 3.7464
Epoch 2/25
  1/236 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7500 - loss: 0.7238

...

236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2500 - loss: 2.2879 - val_accuracy: 0.5823 - val_loss: 2.6671
Epoch 21/25
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6343 - loss: 1.0462 - val_accuracy: 0.5992 - val_loss: 2.7490
Epoch 22/25
  1/236 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.5000 - loss: 1.3744

2024-09-05 01:02:40.190292: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2761561961725852182
2024-09-05 01:02:40.190427: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 17761867372476183312


236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5000 - loss: 1.3744 - val_accuracy: 0.5949 - val_loss: 2.7411
Epoch 23/25
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6145 - loss: 1.0776 - val_accuracy: 0.5696 - val_loss: 2.7846
Epoch 24/25
  1/236 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.5000 - loss: 1.1777

2024-09-05 01:02:46.742433: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2761561961725852182
2024-09-05 01:02:46.742578: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 17761867372476183312


236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5000 - loss: 1.1777 - val_accuracy: 0.5781 - val_loss: 2.7992
Epoch 25/25
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6435 - loss: 1.0076 - val_accuracy: 0.5823 - val_loss: 2.7062


In [8]:

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model with data augmentation
# batch_size = 16
# model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
#           steps_per_epoch=len(X_train) // batch_size,
#           epochs=25,
#           validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.5579 - loss: 2.5601
Test accuracy: 0.5823


: 